In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import eda

from tqdm import tqdm

# os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id']

feat_no = '107_his_'

### Data Load

In [13]:
def elo_load_data(filename=''):
    
    if len(filename):
        df = utils.read_df_pkl(path=f'../input/{filename}*.p')
        return df
    # read pickle
    path_list = glob.glob("../input/*.p")
    for path in path_list:
        filename = re.search(r'/([^/.]*).gz', path).group(1)
        df = utils.read_df_pkl(path=f'../input/{filename}*.p')
train = elo_load_data('train')
test = elo_load_data('test')
train_id = train[key].values
test_id = test[key].values
train_test = pd.concat([train, test], axis=0).reset_index(drop=True)
history = elo_load_data('historical')

100%|██████████| 3/3 [00:00<00:00, 176.09it/s]


#### New Transactionsの最少粒度は下記

In [118]:
print(new.shape)
cols = [key, 'purchase_date', 'purchase_amount', 'merchant_id', 'merchant_category_id', 'installments']
print(new[cols].drop_duplicates().shape)

(1963031, 14)
(1963031, 6)


SystemExit: 

### Cleansing

In [3]:
df = history
# 999という値が1つだけあるので欠損値扱いで-1にする
df['installments'] = df['installments'].where(df['installments']<20, -1)
df['installments'] = df['installments'].replace(-1, np.nan)
df['authorized_flag'] = df['authorized_flag'].replace(-1, 'NA')
df['merchant_id'] =  df['merchant_id'].fillna('NA')
df['category_1'] =  df['category_1'].fillna('NA')
df['category_2'] =  df['category_2'].fillna('NA')
df['category_3'] =  df['category_3'].fillna('NA')

### card_idの粒度で色々集計する
有効かわからないからとりあえず書いてやってみる

### nunique

In [4]:
df = history
feat_nunique = []
nuq_city_id = df.groupby(key)['city_id'].nunique().rename('city_id_nuq@card_id')
feat_nunique.append(nuq_city_id)

nuq_merchant_category_id = df.groupby(key)['merchant_category_id'].nunique().rename('merchant_category_id_nuq@card_id')
feat_nunique.append(nuq_merchant_category_id)

nuq_merchant_id = df.groupby(key)['merchant_id'].nunique().rename('merchant_id_nuq@card_id')
feat_nunique.append(nuq_merchant_id)

nuq_state_id = df.groupby(key)['state_id'].nunique().rename('state_id_nuq@card_id')
feat_nunique.append(nuq_state_id)

nuq_subsector_id = df.groupby(key)['subsector_id'].nunique().rename('subsector_id_nuq@card_id')
feat_nunique.append(nuq_subsector_id)

nuq_category_1 = df.groupby(key)['category_1'].nunique().rename('category_1_nuq@card_id')
feat_nunique.append(nuq_category_1)

nuq_category_2 = df.groupby(key)['category_2'].nunique().rename('category_2_nuq@card_id')
feat_nunique.append(nuq_category_2)

nuq_category_3 = df.groupby(key)['category_3'].nunique().rename('category_3_nuq@card_id')
feat_nunique.append(nuq_category_3)

nuq_installments = df.groupby(key)['installments'].nunique().rename('installments_nuq@card_id')
feat_nunique.append(nuq_installments)

nuq_purchase_date = df.groupby(key)['purchase_date'].nunique().rename('purchase_date_nuq@card_id')
feat_nunique.append(nuq_purchase_date)

nuq_month_lag = df.groupby(key)['month_lag'].nunique().rename('month_lag_nuq@card_id')
feat_nunique.append(nuq_month_lag)

### 作成したnunique featureをjoin

In [14]:
train_test.set_index(key, inplace=True)
for feat in feat_nunique:
    train_test = train_test.join(feat, how='left')

In [15]:
train_test.head()

,feature_1,feature_2,feature_3,first_active_month,target,city_id_nuq@card_id,merchant_category_id_nuq@card_id,merchant_id_nuq@card_id,state_id_nuq@card_id,subsector_id_nuq@card_id,category_1_nuq@card_id,category_2_nuq@card_id,category_3_nuq@card_id,installments_nuq@card_id,purchase_date_nuq@card_id,month_lag_nuq@card_id
card_id,,,,,,,,,,,,,,,,
C_ID_92a2005557,5,2,1,2017-06,-0.820283,7,41,95,3,21,1,2,2,2,259,9
C_ID_3d0044924f,4,1,0,2017-01,0.392913,9,57,142,3,24,2,2,3,8,349,13
C_ID_d639edf6cd,2,2,0,2016-08,0.688056,5,8,13,2,7,1,2,1,1,43,12
C_ID_186d6a6901,4,3,0,2017-09,0.142495,7,25,50,5,13,2,3,3,3,77,6
C_ID_cdbd2c0db2,1,3,0,2017-11,-0.159749,6,26,67,6,17,2,4,2,4,133,4


### month_lag別の集計

In [16]:
feat_ins = history.groupby(key)['installments'].agg({
    'installments_max@card_id':'max'
    ,'installments_min@card_id':'min'
    ,'installments_mean@card_id':'mean'
    ,'installments_std@card_id':'std'
    ,'installments_skew@card_id':'skew'
})

feat_amt = history.groupby(key)['purchase_amount'].agg({
    'purchase_amount_max@card_id':'max'
    ,'purchase_amount_min@card_id':'min'
    ,'purchase_amount_mean@card_id':'mean'
    ,'purchase_amount_std@card_id':'std'
    ,'purchase_amount_skew@card_id':'skew'
})

### aggregate feature setをjoin

In [17]:
for feat in [feat_ins, feat_amt]:
    train_test = train_test.join(feat, how='left')

### Save

In [18]:
train = train_test.loc[train_id, :]
test = train_test.loc[test_id, :]
for col in train.columns:
    if col.count('@'):
        utils.to_pkl_gzip(obj = train[col].values, path=f'../features/1_first_valid/{feat_no}train_his_{col}')
        utils.to_pkl_gzip(obj = test[col].values, path=f'../features/1_first_valid/{feat_no}test_his_{col}')